In [1]:
import os
import subprocess
import pandas as pd
import platform
import pathlib
import numpy as np
from annotations import *
from extract_video_features import *
from extract_audio_features import *
import cv2
import seaborn as sns
from scipy.stats import kurtosis, skew
from sklearn.metrics import f1_score
import importlib
from tqdm import tqdm
import ordinal_classification as o_c
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from util import runGridSearchClassifiers


In [2]:
#get current directory
cwd = os.getcwd()

In [3]:
cwd

'/Users/valentinadiproietto/filrouge'

In [4]:
OpenFace_folder = '/Users/valentinadiproietto/OpenFace'
filename_annotations = 'https://docs.google.com/spreadsheets/d/1Rqu1sJiD-ogc4a6R491JTiaYacptOTqh6DKqhwTa8NA/gviz/tq?tqx=out:csv&sheet=Template'

In [5]:
Video_folder = '/Users/valentinadiproietto/Desktop/video_stress'

In [6]:
video_paths, video_names = get_videos(Video_folder)

In [7]:
video_names

['WIN_20210331_21_22_52_Pro',
 'WIN_20210329_14_13_45_Pro',
 'WIN_20210406_18_49_10_Pro',
 'WIN_20210408_14_11_32_Pro',
 'WIN_20210408_15_20_51_Pro',
 'WIN_20210404_10_58_27_Pro',
 'WIN_20210414_06_24_52_Pro',
 'WIN_20210406_15_06_15_Pro',
 'WIN_20210417_14_53_12_Pro',
 'WIN_20210413_15_38_01_Pro',
 'WIN_20210408_11_48_58_Pro',
 'WIN_20210408_16_04_32_Pro',
 'WIN_20210329_10_16_02_Pro',
 'WIN_20210323_19_17_40_Pro',
 'WIN_20210409_10_26_11_Pro',
 'Test_pour_AFPA',
 'WIN_20210405_15_09_16_Pro',
 'WIN_20210407_14_54_56_Pro_edit2',
 'WIN_20210406_21_05_52_Pro',
 'WIN_20210403_18_49_15_Pro',
 'WIN_20210408_14_02_19_Pro',
 'WIN_20210415_15_41_24_Pro',
 'WIN_20210406_18_35_52_Pro',
 'WIN_20210402_14_27_50_Pro',
 'WIN_20210407_09_04_05_Pro',
 'WIN_20210402_19_04_53_Pro',
 'WIN_20210416_08_06_54_Pro',
 'Video_1',
 'WIN_20210408_14_00_44_Pro',
 'WIN_20210404_21_41_12_Pro',
 'WIN_20210330_13_10_29_Pro']

In [8]:
list_dataframes = []
for i in video_names: 
    list_dataframes.append(create_dataframe_video('/Users/valentinadiproietto/OpenFace/processed/', i))


In [9]:
video_names.remove('WIN_20210329_14_13_45_Pro')
video_names.remove('WIN_20210402_14_27_50_Pro')


In [10]:
len(video_names)

29

In [11]:
list_df_max = []
for v_name in tqdm(video_names):
    df_annoted = get_df_video_with_annotations('/Users/valentinadiproietto/OpenFace/processed/', v_name, filename_annotations, "max")
    list_df_max.append(eliminate_features(df_annoted))


    

100%|██████████| 29/29 [02:50<00:00,  5.87s/it]


In [12]:
list_df_max[0].columns

Index(['frame', 'face_id', 'timestamp', 'confidence', 'success', 'AU01_r',
       'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r',
       'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r',
       'AU26_r', 'AU45_r', 'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c',
       'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c',
       'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c', 'gaze_0_x',
       'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z',
       'gaze_angle_x', 'gaze_angle_y', 'pose_Tx', 'pose_Ty', 'pose_Tz',
       'pose_Rx', 'pose_Ry', 'pose_Rz', 'type_candidat', 'sexe', 'video_name',
       'stress_global', 'stress', 'diapo'],
      dtype='object')

##

## AGGREGATION TIME WINDOW AND FIRST RF ON TIME WINDOW %

In [12]:
df_with_deriv = []
for i in list_df_max:
    to_drop = ['frame','face_id','timestamp','confidence','success', 'type_candidat']
    i = add_frameTimeWindow(i)
    i = i.drop(to_drop, axis = 1)
    i = add_derivatives_drop_spatial(i)
    df_with_deriv.append(i)

In [22]:
spatial_features = ['gaze_0_x_de_mean',
        'gaze_0_x_de_de_mean', 
       'gaze_0_y_de_mean', 'gaze_0_y_de_de_mean',
        'gaze_0_z_de_mean', 
       'gaze_0_z_de_de_mean', 'gaze_1_x_de_mean',
       'gaze_1_x_de_de_mean', 
       'gaze_1_y_de_mean',  'gaze_1_y_de_de_mean',
        'gaze_1_z_de_mean',
       'gaze_1_z_de_de_mean',  'gaze_angle_x_de_mean',
        'gaze_angle_x_de_de_mean',
        'gaze_angle_y_de_mean', 
       'gaze_angle_y_de_de_mean', 'pose_Tx_de_mean',
       'pose_Tx_de_de_mean', 
       'pose_Ty_de_mean',  'pose_Ty_de_de_mean',
        'pose_Tz_de_mean', 
       'pose_Tz_de_de_mean',  'pose_Rx_de_mean',
       'pose_Rx_de_de_mean', 
       'pose_Ry_de_mean', 'pose_Ry_de_de_mean',
       'pose_Rz_de_mean', 
       'pose_Rz_de_de_mean']

In [27]:
groupby_features= ['video_name','stress_global','stress','frameTimeWindow','sexe', 'diapo']

df_total = pd.concat(df_with_deriv)


In [28]:
df_total = df_total.groupby(groupby_features).agg(['mean']).reset_index()
df_total.columns= df_total.columns.map('_'.join).str.strip('_')
df_total[['stress']].value_counts()

stress
1.0       2419
0.0       2064
2.0        638
3.0         20
dtype: int64

In [29]:
pd.options.display.max_seq_items = 2000
df_total = df_total.drop(spatial_features, axis =1)
df_total.columns

Index(['video_name', 'stress_global', 'stress', 'frameTimeWindow', 'sexe',
       'diapo', 'AU01_r_mean', 'AU02_r_mean', 'AU04_r_mean', 'AU05_r_mean',
       'AU06_r_mean', 'AU07_r_mean', 'AU09_r_mean', 'AU10_r_mean',
       'AU12_r_mean', 'AU14_r_mean', 'AU15_r_mean', 'AU17_r_mean',
       'AU20_r_mean', 'AU23_r_mean', 'AU25_r_mean', 'AU26_r_mean',
       'AU45_r_mean', 'AU01_c_mean', 'AU02_c_mean', 'AU04_c_mean',
       'AU05_c_mean', 'AU06_c_mean', 'AU07_c_mean', 'AU09_c_mean',
       'AU10_c_mean', 'AU12_c_mean', 'AU14_c_mean', 'AU15_c_mean',
       'AU17_c_mean', 'AU20_c_mean', 'AU23_c_mean', 'AU25_c_mean',
       'AU26_c_mean', 'AU28_c_mean', 'AU45_c_mean', 'AU01_r_de_mean',
       'AU01_r_de_de_mean', 'AU02_r_de_mean', 'AU02_r_de_de_mean',
       'AU04_r_de_mean', 'AU04_r_de_de_mean', 'AU05_r_de_mean',
       'AU05_r_de_de_mean', 'AU06_r_de_mean', 'AU06_r_de_de_mean',
       'AU07_r_de_mean', 'AU07_r_de_de_mean', 'AU09_r_de_mean',
       'AU09_r_de_de_mean', 'AU10_r_de_mean', 'A

In [30]:
x = df_total.drop(['video_name','stress_global', 'stress','frameTimeWindow'], axis = 1)
x.shape

(5141, 71)

In [31]:
#Replace string values
x.sexe = x.sexe.replace('H',0)
x.sexe = x.sexe.replace('F',1)

y = df_total[['stress']]
y.shape

(5141, 1)

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut

models_list = [RandomForestClassifier(random_state = 42, n_jobs=-1)]
parameters_list = [
                {'n_estimators': [100, 150, 200, 250, 300], 'max_depth':[10, 15, 20, 25,30], 'class_weight':[None,'balanced']}
                ]
groups = df_total['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(x, y, groups)

In [33]:
best_result, y_predict, result_list = runGridSearchClassifiers(x, y, cv_loo, models_list, parameters_list, output_predict=True, n_jobs=-1, verbose=True)

Fitting 29 folds for each of 50 candidates, totalling 1450 fits
Best estimator RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=200,
                       n_jobs=-1, random_state=42)
Best results 0.48680457189978915
Best params {'class_weight': 'balanced', 'max_depth': 10, 'n_estimators': 200}
accuracy (mean, std) 0.508148691748685 0.19044369172224368
f1 (mean, std) 0.48680457189978915 0.20394140360004606
balanced accuracy (mean, std) 0.4782190877946046 0.16339511752779573
precision (mean, std) 0.5884394323901357 0.22265876772921822
recall (mean, std) 0.508148691748685 0.19044369172224368



In [34]:
# Saving predictions
predict5s = pd.concat([df_total[['video_name','diapo','frameTimeWindow']],
                        pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)


In [35]:
best_result

{'best_estimator': RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=200,
                        n_jobs=-1, random_state=42),
 'best_score': 0.48680457189978915,
 'best_params': {'class_weight': 'balanced',
  'max_depth': 10,
  'n_estimators': 200},
 'mean_test_f1_score': 0.48680457189978915,
 'std_test_f1_score': 0.20394140360004606,
 'mean_test_accuracy_score': 0.508148691748685,
 'std_test_accuracy_score': 0.19044369172224368,
 'mean_test_balanced_accuracy_score': 0.4782190877946046,
 'std_test_balanced_accuracy_score': 0.16339511752779573,
 'mean_test_precision': 0.5884394323901357,
 'std_test_precision': 0.22265876772921822,
 'mean_test_recall': 0.508148691748685,
 'std_test_recall': 0.19044369172224368}

In [36]:
predict5s.to_csv('au_tw5.csv')

In [37]:
predict5s

,video_name,diapo,frameTimeWindow,ypredict
0,Test_pour_AFPA,9,44,1.0
1,Test_pour_AFPA,9,45,0.0
2,Test_pour_AFPA,9,46,1.0
3,Test_pour_AFPA,9,47,1.0
4,Test_pour_AFPA,9,48,1.0
...,...,...,...,...
5136,WIN_20210417_14_53_12_Pro,18,120,1.0
5137,WIN_20210417_14_53_12_Pro,18,121,1.0
5138,WIN_20210417_14_53_12_Pro,18,122,1.0
5139,WIN_20210417_14_53_12_Pro,18,123,1.0


## PIVOT TABLE WITH COUNT OF PREDICTION 5SEC, TO PREDICT STRESS BY DIAPO

In [38]:
X = predict5s.pivot_table(values='frameTimeWindow', columns='ypredict', index=['video_name','diapo'], aggfunc='count', fill_value=0)
X_sum = X.sum(axis=1).values.copy()
for col_number in range(len(X.columns)):
    X.iloc[:,col_number] = X.iloc[:,col_number] / X_sum
X
 

ypredict                              0.0       1.0       2.0
video_name                diapo                              
Test_pour_AFPA            1      0.647059  0.352941  0.000000
                          8      0.300000  0.700000  0.000000
                          9      0.100000  0.900000  0.000000
                          10     0.200000  0.800000  0.000000
                          11     0.200000  0.800000  0.000000
...                                   ...       ...       ...
WIN_20210417_14_53_12_Pro 10     0.928571  0.071429  0.000000
                          11     0.950000  0.050000  0.000000
                          12     0.666667  0.333333  0.000000
                          17     0.428571  0.571429  0.000000
                          18     0.043478  0.913043  0.043478

[232 rows x 3 columns]

In [39]:
#STRESS BY DIAPO

df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.drop(df_annotations_stress.columns[-1],axis=1).set_index('video_name').stack()
df_annotations_stress = pd.DataFrame(df_annotations_stress).reset_index()
df_annotations_stress.columns = ['video_name','diapo','stress']
#df_annotations_stress = df_annotations_stress.set_index(['video_name','diapo'])
df_annotations_stress['diapo'] = df_annotations_stress['diapo'].astype(int)
df_annotations_stress

,video_name,diapo,stress
0,Test_pour_AFPA,1,1.0
1,Test_pour_AFPA,8,1.0
2,Test_pour_AFPA,9,0.0
3,Test_pour_AFPA,10,0.0
4,Test_pour_AFPA,11,0.0
...,...,...,...
235,WIN_20210417_14_53_12_Pro,10,0.0
236,WIN_20210417_14_53_12_Pro,11,0.0
237,WIN_20210417_14_53_12_Pro,12,0.0
238,WIN_20210417_14_53_12_Pro,17,1.0


In [40]:
Xy = X.merge(df_annotations_stress, how='inner', on=['video_name','diapo'])
X = Xy.iloc[:,:-1].set_index(['video_name','diapo'])
y = Xy.iloc[:,-1]
Xy

,video_name,diapo,0.0,1.0,2.0,stress
0,Test_pour_AFPA,1,0.647059,0.352941,0.000000,1.0
1,Test_pour_AFPA,8,0.300000,0.700000,0.000000,1.0
2,Test_pour_AFPA,9,0.100000,0.900000,0.000000,0.0
3,Test_pour_AFPA,10,0.200000,0.800000,0.000000,0.0
4,Test_pour_AFPA,11,0.200000,0.800000,0.000000,0.0
...,...,...,...,...,...,...
227,WIN_20210417_14_53_12_Pro,10,0.928571,0.071429,0.000000,0.0
228,WIN_20210417_14_53_12_Pro,11,0.950000,0.050000,0.000000,0.0
229,WIN_20210417_14_53_12_Pro,12,0.666667,0.333333,0.000000,0.0
230,WIN_20210417_14_53_12_Pro,17,0.428571,0.571429,0.000000,1.0


In [41]:
X

0.0       1.0       2.0
video_name                diapo                              
Test_pour_AFPA            1      0.647059  0.352941  0.000000
                          8      0.300000  0.700000  0.000000
                          9      0.100000  0.900000  0.000000
                          10     0.200000  0.800000  0.000000
                          11     0.200000  0.800000  0.000000
...                                   ...       ...       ...
WIN_20210417_14_53_12_Pro 10     0.928571  0.071429  0.000000
                          11     0.950000  0.050000  0.000000
                          12     0.666667  0.333333  0.000000
                          17     0.428571  0.571429  0.000000
                          18     0.043478  0.913043  0.043478

[232 rows x 3 columns]

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [43]:
best_result_diapo, y_predict_diapo, result_list_diapo = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list,output_predict=True, n_jobs=-1, verbose=True)

Fitting 29 folds for each of 20 candidates, totalling 580 fits
Best estimator LogisticRegression(C=10, multi_class='multinomial', random_state=42)
Best results 0.47966477966477966
Best params {'C': 10, 'class_weight': None}
accuracy (mean, std) 0.521551724137931 0.2345873245549617
f1 (mean, std) 0.47966477966477966 0.2599031158998486
balanced accuracy (mean, std) 0.513232074438971 0.2373934026488492
precision (mean, std) 0.5306855500821018 0.3123271557030385
recall (mean, std) 0.521551724137931 0.2345873245549617

Fitting 29 folds for each of 80 candidates, totalling 2320 fits
Best estimator Pipeline(steps=[('pca', PCA(n_components=2)),
                ('logistic',
                 LogisticRegression(C=10, multi_class='multinomial'))])
Best results 0.47966477966477966
Best params {'logistic__C': 10, 'logistic__class_weight': None, 'pca__n_components': 2}
accuracy (mean, std) 0.521551724137931 0.2345873245549617
f1 (mean, std) 0.47966477966477966 0.2599031158998486
balanced accuracy (me

In [44]:
best_result_diapo

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=1)),
                 ('knn', KNeighborsClassifier(n_neighbors=7, p=1))]),
 'best_score': 0.5376745381055725,
 'best_params': {'knn__n_neighbors': 7,
  'knn__p': 1,
  'knn__weights': 'uniform',
  'pca__n_components': 1},
 'mean_test_f1_score': 0.5376745381055725,
 'std_test_f1_score': 0.22699517173771702,
 'mean_test_accuracy_score': 0.5560344827586207,
 'std_test_accuracy_score': 0.19305341100481632,
 'mean_test_balanced_accuracy_score': 0.5548850574712643,
 'std_test_balanced_accuracy_score': 0.1864526144951227,
 'mean_test_precision': 0.5847393267651888,
 'std_test_precision': 0.28408637772660134,
 'mean_test_recall': 0.5560344827586207,
 'std_test_recall': 0.19305341100481632}

In [45]:
y_predict_diapo
##ATTENZIONE NON PREDICE MAI I 2

array([1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
       0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
       1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 0.

## ORA SEMPRE PARTENDO DALLE PEDIZIONI DI  5 SECONDI FACCIO NEL AGGIUNGENDO MINMAX sulle predizioni e predico su diapo

In [46]:
predict5s['ypredict'].value_counts()

1.0    3302
0.0    1670
2.0     169
Name: ypredict, dtype: int64

In [47]:
predict5s

,video_name,diapo,frameTimeWindow,ypredict
0,Test_pour_AFPA,9,44,1.0
1,Test_pour_AFPA,9,45,0.0
2,Test_pour_AFPA,9,46,1.0
3,Test_pour_AFPA,9,47,1.0
4,Test_pour_AFPA,9,48,1.0
...,...,...,...,...
5136,WIN_20210417_14_53_12_Pro,18,120,1.0
5137,WIN_20210417_14_53_12_Pro,18,121,1.0
5138,WIN_20210417_14_53_12_Pro,18,122,1.0
5139,WIN_20210417_14_53_12_Pro,18,123,1.0


In [48]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

In [49]:
# Autre méthode
X_diapo_minmax = predict5s.groupby(['video_name','diapo']).agg({'ypredict':['mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew]})

In [50]:
X_diapo_minmax

ypredict                             \
                                     mean  min  max median       std   
video_name                diapo                                        
Test_pour_AFPA            1      0.352941  0.0  1.0    0.0  0.485071   
                          8      0.700000  0.0  1.0    1.0  0.483046   
                          9      0.900000  0.0  1.0    1.0  0.307794   
                          10     0.800000  0.0  1.0    1.0  0.414039   
                          11     0.800000  0.0  1.0    1.0  0.410391   
...                                   ...  ...  ...    ...       ...   
WIN_20210417_14_53_12_Pro 10     0.071429  0.0  1.0    0.0  0.267261   
                          11     0.050000  0.0  1.0    0.0  0.223607   
                          12     0.333333  0.0  1.0    0.0  0.478714   
                          17     0.571429  0.0  1.0    1.0  0.534522   
                          18     1.000000  0.0  2.0    1.0  0.301511   

                                                                              
                                percentil25 percentil75   kurtosis      skew  
video_name                diapo                                               
Test_pour_AFPA            1            0.00         1.0  -1.621212  0.615457  
                          8            0.25         1.0  -1.238095 -0.872872  
                          9            1.00         1.0   5.111111 -2.666667  
                          10           1.00         1.0   0.250000 -1.500000  
                          11           1.00         1.0   0.250000 -1.500000  
...                                     ...         ...        ...       ...  
WIN_20210417_14_53_12_Pro 10           0.00         0.0   9.076923  3.328201  
                          11           0.00         0.0  15.052632  4.129483  
                          12           0.00         1.0  -1.500000  0.707107  
                          17           0.00         1.0  -1.916667 -0.288675  
                          18           1.00         1.0   8.500000  0.000000  

[232 rows x 9 columns]

In [51]:
#questo e' lo stress vero per diapositiva
y

0      1.0
1      1.0
2      0.0
3      0.0
4      0.0
      ... 
227    0.0
228    0.0
229    0.0
230    1.0
231    1.0
Name: stress, Length: 232, dtype: float64

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X_diapo_minmax.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X_diapo_minmax, y, groups)

In [53]:
best_result_diapo_minmax, y_predict_diapo_minmax, result_list_diapo = runGridSearchClassifiers(X_diapo_minmax, y, cv_loo, models_list, parameters_list,output_predict=True, n_jobs=-1, verbose=True)

Fitting 29 folds for each of 20 candidates, totalling 580 fits
Best estimator LogisticRegression(C=0.05, multi_class='multinomial', random_state=42)
Best results 0.4238218677873851
Best params {'C': 0.05, 'class_weight': None}
accuracy (mean, std) 0.45689655172413796 0.24184069205354372
f1 (mean, std) 0.4238218677873851 0.25766734564927196
balanced accuracy (mean, std) 0.4841269841269841 0.2570479622113761
precision (mean, std) 0.495920566502463 0.3250784142054801
recall (mean, std) 0.45689655172413796 0.24184069205354372

Fitting 29 folds for each of 80 candidates, totalling 2320 fits
Best estimator Pipeline(steps=[('pca', PCA(n_components=3)),
                ('logistic',
                 LogisticRegression(C=4, multi_class='multinomial'))])
Best results 0.4181390448631828
Best params {'logistic__C': 4, 'logistic__class_weight': None, 'pca__n_components': 3}
accuracy (mean, std) 0.4525862068965517 0.2376560129670728
f1 (mean, std) 0.4181390448631828 0.2526499448773479
balanced accura

In [54]:
best_result_diapo_minmax

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=3)),
                 ('knn', KNeighborsClassifier(n_neighbors=6, p=1))]),
 'best_score': 0.5311058194678884,
 'best_params': {'knn__n_neighbors': 6,
  'knn__p': 1,
  'knn__weights': 'uniform',
  'pca__n_components': 3},
 'mean_test_f1_score': 0.5311058194678884,
 'std_test_f1_score': 0.23046659357279956,
 'mean_test_accuracy_score': 0.5301724137931034,
 'std_test_accuracy_score': 0.22422080695930954,
 'mean_test_balanced_accuracy_score': 0.46923918992884517,
 'std_test_balanced_accuracy_score': 0.2239854160573625,
 'mean_test_precision': 0.589054802955665,
 'std_test_precision': 0.25619859058454536,
 'mean_test_recall': 0.5301724137931034,
 'std_test_recall': 0.22422080695930954}

In [55]:
#è meglio arriva a predire i 2
y_predict_diapo_minmax

array([1., 0., 1., 1., 1., 1., 0., 0., 0., 2., 0., 0., 0., 1., 1., 0., 0.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 0., 2., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 2., 0., 1., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0.

In [56]:
# Saving predictions
df_diapo_minmax = pd.concat([X_diapo_minmax.reset_index()[['video_name','diapo']],
                        pd.DataFrame(y_predict_diapo_minmax, columns=['ypredict'])],axis=1)

df_diapo_minmax.columns = ['video_name', 'diapo', 'ypredict']
df_diapo_minmax
df_diapo_minmax.to_csv('au_tw5_diapo_minmax.csv')

## GLOBAL STRESS, UTILIZZANDO LE PREDIZIONI DI DIAPO MINMAX

In [57]:
stress_diapo_by_video = df_diapo_minmax.pivot_table(values='ypredict', columns='diapo',index='video_name',aggfunc='mean')
stress_diapo_by_video

diapo,1,8,9,10,11,12,17,18
video_name,,,,,,,,
Test_pour_AFPA,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
Video_1,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0
WIN_20210323_19_17_40_Pro,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
WIN_20210329_10_16_02_Pro,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
WIN_20210330_13_10_29_Pro,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
WIN_20210331_21_22_52_Pro,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
WIN_20210402_19_04_53_Pro,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
WIN_20210403_18_49_15_Pro,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
WIN_20210404_10_58_27_Pro,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [58]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.set_index(['video_name'])
df_annotations_stress

,1,8,9,10,11,12,17,18,stress_global
video_name,,,,,,,,,
Test_pour_AFPA,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
Video_1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
WIN_20210323_19_17_40_Pro,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
WIN_20210329_10_16_02_Pro,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
WIN_20210330_13_10_29_Pro,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
WIN_20210331_21_22_52_Pro,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
WIN_20210402_14_27_50_Pro,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
WIN_20210402_19_04_53_Pro,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0
WIN_20210403_18_49_15_Pro,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0


In [59]:
Xy = stress_diapo_by_video.merge(df_annotations_stress.iloc[:,-1],how='inner',on='video_name')
X_global= Xy.iloc[:,:-1]
y_global = Xy.iloc[:,-1]

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X_global.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X_global, y_global, groups)

In [61]:
best_result_tw5_diapo_minmax_global, y_predict_tw5_diapo_minmax_global, result_list = runGridSearchClassifiers(X_global, y_global, cv_loo, models_list, parameters_list, output_predict=True, n_jobs=-1, verbose=True)

Fitting 29 folds for each of 20 candidates, totalling 580 fits
Best estimator LogisticRegression(C=1, multi_class='multinomial', random_state=42)
Best results 0.5172413793103449
Best params {'C': 1, 'class_weight': None}
accuracy (mean, std) 0.5172413793103449 0.4997026464203254
f1 (mean, std) 0.5172413793103449 0.4997026464203254
balanced accuracy (mean, std) 0.5172413793103449 0.4997026464203254
precision (mean, std) 0.5172413793103449 0.4997026464203254
recall (mean, std) 0.5172413793103449 0.4997026464203254

Fitting 29 folds for each of 80 candidates, totalling 2320 fits
Best estimator Pipeline(steps=[('pca', PCA(n_components=1)),
                ('logistic',
                 LogisticRegression(C=0.01, class_weight='balanced',
                                    multi_class='multinomial'))])
Best results 0.5172413793103449
Best params {'logistic__C': 0.01, 'logistic__class_weight': 'balanced', 'pca__n_components': 1}
accuracy (mean, std) 0.5172413793103449 0.49970264642032547
f1 (

In [62]:
best_result_tw5_diapo_minmax_global

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=2)),
                 ('knn', KNeighborsClassifier(n_neighbors=10, p=1))]),
 'best_score': 0.6206896551724138,
 'best_params': {'knn__n_neighbors': 10,
  'knn__p': 1,
  'knn__weights': 'uniform',
  'pca__n_components': 2},
 'mean_test_f1_score': 0.6206896551724138,
 'std_test_f1_score': 0.48521542343000995,
 'mean_test_accuracy_score': 0.6206896551724138,
 'std_test_accuracy_score': 0.48521542343000995,
 'mean_test_balanced_accuracy_score': 0.6206896551724138,
 'std_test_balanced_accuracy_score': 0.48521542343000995,
 'mean_test_precision': 0.6206896551724138,
 'std_test_precision': 0.48521542343000995,
 'mean_test_recall': 0.6206896551724138,
 'std_test_recall': 0.48521542343000995}

In [63]:
y_predict_tw5_diapo_minmax_global

array([0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.])

In [64]:
#seeing prediction
df_global_5tw_minmax = pd.concat([X_global.reset_index()[['video_name']],
                        pd.DataFrame(y_predict_tw5_diapo_minmax_global, columns=['y_predict_tw5_diapo_minmax_global'])],axis=1)

df_global_5tw_minmax.columns = ['video_name', 'y_predict_tw5_diapo_minmax_global']
df_global_5tw_minmax


,video_name,y_predict_tw5_diapo_minmax_global
0,Test_pour_AFPA,0.0
1,Video_1,0.0
2,WIN_20210323_19_17_40_Pro,1.0
3,WIN_20210329_10_16_02_Pro,1.0
4,WIN_20210330_13_10_29_Pro,0.0
5,WIN_20210331_21_22_52_Pro,1.0
6,WIN_20210402_19_04_53_Pro,1.0
7,WIN_20210403_18_49_15_Pro,1.0
8,WIN_20210404_10_58_27_Pro,1.0
9,WIN_20210404_21_41_12_Pro,0.0


In [65]:
#SAving predictions
df_global_5tw_minmax.to_csv('au_tw5_diapo_minmax_global.csv')